In [31]:
import pandas as pd
import requests
import os 
import ast
import logging
import time
from time import sleep
import json

In [32]:
df_fmc = pd.read_csv("df_fmc.csv")
df_fmc = df_fmc[['title', 'author', 'fmc_name']]
df_sample = df_fmc.sample(n=100, random_state=42)
df_sample

,title,author,fmc_name
1753,Its first loveJust like the last one,"Garg, Sachin",Kanika Merchant
259,Vipers Tangle,"Mauriac, François",Isa Fondaudège
2072,Kiss of Steel London Steampunk 1,"McMaster, Bec",Honoria Todd
1000,The Jerusalem Inception A young talmudic Schol...,"Azrieli, Avraham",Lottie
56,The Circle Trilogy The Circle Trilogy 13,"Roberts, Nora",Glenna
...,...,...,...
2121,Covet The Clann 2,"Darnell, Melissa",Savannah Colbert
906,Reason to Breathe Breathing 1,"Donovan, Rebecca",Emma Thomas
789,The Magic Finger,"Dahl, Roald",NaN
438,The Time of My Life,"Ahern, Cecelia",Lucy Silchester


In [33]:
system_prompt = """
    You are a helpful literary assistant. Your job is to search the internet and assess several aspects for books that are provided to you.

    You return a Python list for each book, where each element is the answer to a question.
    The question are independent of each other, so if one question is true, the opposite can also be true. 
    The list always needs to have EXACTLY 29 elements.
    Most of the questions are about a female major character of the book whose name will be provided alongside the book title. 
    For all questions about the female character, consider any occurrence of the trait or behavior at any point in the book. 

    1. What's the gender of the author? (0 = male, 1 = female, 99 = unclear or non-binary) 
    2. In what year was the book published? (numeric) 
    3. (As a list) What is/are the genre(s) of the book? (list of a maximum of 5 genres in the order of their relevance, try to use common genre names, lower case only) 
    4. Is the female character saved by a male character? (0 = no, 1 = yes) 
    5. Does the female character save a male character? (0 = no, 1 = yes) 
    6. Is the female character protected by a male character? (0 = no, 1 = yes) 
    7. Does the female character protect a male character? (0 = no, 1 = yes) 
    8. Is the female character’s problem solved through help or luck? (0 = no, 1 = yes)  
    9. Does the female character solve her own problem through skill? (0 = no, 1 = yes) 
    10. Is the female character victimized/harmed by a male character? (0 = no, 1 = yes) 
    11. Is the female character a perpetrator or does she harm a male character? (0 = no, 1 = yes) 
    12. Does the female character follow orders? (0 = no, 1 = yes) 
    13. Does the female character give orders? (0 = no, 1 = yes) 
    14. Is the female character physically weak/incapable? (0 = no, 1 = yes) 
    15. Is the female character physically strong/capable? (0 = no, 1 = yes) 
    16. Is the female character fearful/scared of taking risks/challenges? (0 = no, 1 = yes) 
    17. Is the female character brave/likes taking risks/challenges? (0 = no, 1 = yes)  
    18: Does the female character care for others (e.g., nurses someone, comforts someone, takes care of children or sick)? (0 = no, 1 = yes)  
    19: Does the female character not engage in caregiving behaviors (e.g., focuses on tasks or goals without providing emotional or physical support to others)? (0 = no, 1 = yes) 
    20: Is the female character emotionally aware and attuned to others' needs, moods, or the dynamics between people, and tries to maintain harmony? (0 = no, 1 = yes)  
    21: Does the female character focus more on practical matters or impersonal concerns than others’ emotions or social dynamics? (0 = no, 1 = yes)  
    22: Does the female character sacrifice her own goals, desires, or needs to prioritize someone else’s wellbeing or to avoid conflict? (0 = no, 1 = yes)  
    23: Does the female character maintain personal goals and boundaries, even when others express emotional needs or expectations? (0 = no, 1 = yes)  
    24. Is the female character a homemaker? (0 = no, 1 = yes)  
    25. Is the female character a breadwinner? (0 = no, 1 = yes)  
    26. Is the female character admired for her beauty? (0 = no, 1 = yes)  
    27. Is the female character admired for her intelligence? (0 = no, 1 = yes)  
    28. Does the female character have a lower rank occupation (e.g., nurse, assistant, maid, …)? (0 = no, 1 = yes)  
    29. Does the female character have a higher rank occupation (doctor, manager, ruler, …)? (0 = no, 1 = yes)  

    If you cannot find information for a female-character-related question, answer the question with a 0.

    Example output: 
    [1, 2015, ['fantasy', 'young adult'], 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0]
"""

user_prompt_template = """
    Please answer the 29 questions defined in your system prompt for the book {title} by {author}.
    The female character in question is {fmc_name}. IMPORTANT: Only return a Python list with exactly 29 elements — no explanations, no comments, no extra fields.
"""

In [34]:
json_schema = {
    "type": "object",
    "properties": {
        "gender": { "type": "integer", "enum": [0, 1, 99] },
        "year": { "type": "integer" },
        "genres": {
            "type": "array",
            "items": { "type": "string" },
            "minItems": 1,
            "maxItems": 5
        },
        "feature_1": { "type": "integer", "enum": [0, 1] },
        "feature_2": { "type": "integer", "enum": [0, 1] },
        "feature_3": { "type": "integer", "enum": [0, 1] },
        "feature_4": { "type": "integer", "enum": [0, 1] },
        "feature_5": { "type": "integer", "enum": [0, 1] },
        "feature_6": { "type": "integer", "enum": [0, 1] },
        "feature_7": { "type": "integer", "enum": [0, 1] },
        "feature_8": { "type": "integer", "enum": [0, 1] },
        "feature_9": { "type": "integer", "enum": [0, 1] },
        "feature_10": { "type": "integer", "enum": [0, 1] },
        "feature_11": { "type": "integer", "enum": [0, 1] },
        "feature_12": { "type": "integer", "enum": [0, 1] },
        "feature_13": { "type": "integer", "enum": [0, 1] },
        "feature_14": { "type": "integer", "enum": [0, 1] },
        "feature_15": { "type": "integer", "enum": [0, 1] },
        "feature_16": { "type": "integer", "enum": [0, 1] },
        "feature_17": { "type": "integer", "enum": [0, 1] },
        "feature_18": { "type": "integer", "enum": [0, 1] },
        "feature_19": { "type": "integer", "enum": [0, 1] },
        "feature_20": { "type": "integer", "enum": [0, 1] },
        "feature_21": { "type": "integer", "enum": [0, 1] },
        "feature_22": { "type": "integer", "enum": [0, 1] },
        "feature_23": { "type": "integer", "enum": [0, 1] },
        "feature_24": { "type": "integer", "enum": [0, 1] },
        "feature_25": { "type": "integer", "enum": [0, 1] },
        "feature_26": { "type": "integer", "enum": [0, 1] }
    },
    "required": ["gender", "year", "genres", 
                 "feature_1",  "feature_2",  "feature_3",  "feature_4",  "feature_5",  "feature_6",  "feature_7",
                 "feature_8",  "feature_9",  "feature_10",  "feature_11",  "feature_12",  "feature_13",  "feature_14",
                 "feature_15",  "feature_16", "feature_17",  "feature_18",  "feature_19",  "feature_20",  "feature_21", 
                 "feature_22",  "feature_23",  "feature_24",  "feature_25",  "feature_26"]
}

In [35]:
# Set up logging for error tracking
logging.basicConfig(filename='features_log.log', level=logging.INFO)

# Setting API key and headers
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# Endpoint for the API
url = "https://api.perplexity.ai/chat/completions"

# Store responses, citations, and titles + authors for progress
responses = []
citations = []
titles = []
authors = []
fmc_names = []

In [36]:
# Function to send a request to the API
def send_request(title, author, fmc_name, retries=3):

    user_prompt = user_prompt_template.format(title=title, author=author, fmc_name = fmc_name)

    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 2000,
        "temperature": 0,
        "response_format": {
            "type": "json_schema",
            "json_schema": { "schema": json_schema} }
        }

    try:
        # Make the request to the API
        response = requests.post(url, json=payload, headers=headers)
        
        # Check if the response is successful
        if response.status_code == 200:
            response_json = response.json()
            answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
            citation = response_json.get("citations", [])
            return answer, citation
        else:
            raise Exception(f"API call failed with status code {response.status_code}")
    
    except Exception as e:
        # Log the error and retry if retries left
        logging.error(f"Error with request for {title} by {author}: {str(e)}")
        if retries > 0:
            logging.info(f"Retrying {retries} more times...")
            #sleep(60)  # Sleep to avoid hitting rate limits
            return send_request(title, author, fmc_name, retries - 1)
        else:
            logging.error(f"Failed after retries: {title} by {author}")
            return f"Error: {str(e)}", []

In [37]:
def save_progress():
    if titles:  # Only save if there's something new
        progress_df = pd.DataFrame({
            'title': titles,
            'author': authors,
            'fmc_name': fmc_names,
            'answer': responses,
            'citation': citations
        })

        file_exists = os.path.exists('progress_features.csv')

        if file_exists:
            progress_df.to_csv('progress_features.csv', mode='a', header=False, index=False)
            logging.info(f"Progress appended: {len(titles)} books.")
        else:
            progress_df.to_csv('progress_features.csv', index=False)
            logging.info(f"Progress saved: {len(titles)} books.")

        # Clear the lists so next save only writes new data
        titles.clear()
        authors.clear()
        responses.clear()
        citations.clear()
        fmc_names.clear()

In [38]:
start_time = time.time()

# Run for df_subset_1
for index, row in df_sample.iterrows():
    title = row["title"]
    author = row["author"]
    fmc_name = row["fmc_name"]

    # Send request and handle response
    answer, citation = send_request(title, author, fmc_name)
    
    # Store responses
    responses.append(answer)
    citations.append(citation)
    titles.append(title)
    authors.append(author)
    fmc_names.append(fmc_name)

    # Save progress every 10 books
    if (index + 1) % 10 == 0:
        save_progress()

# Save the final progress
save_progress()

# Log the total execution time
end_time = time.time()
execution_time = end_time - start_time
logging.info(f"Completed 100 book requests in {execution_time:.2f} seconds.")

$44.4

100 Bücher später: $43.78
0.62 * 140 = 86.8 für alle Bücher


In [39]:
df_result = pd.read_csv("progress_features.csv")
df_result

,title,author,fmc_name,answer,citation
0,Its first loveJust like the last one,"Garg, Sachin",Kanika Merchant,"{ ""gender"": 0, ""year"": 0, ""genres"": [""romance""...",['https://www.bookchor.com/book/9788192222660/...
1,Vipers Tangle,"Mauriac, François",Isa Fondaudège,"{ ""gender"": 0, ""year"": 1932, ""genres"": [""liter...",['https://www.enotes.com/topics/vipers-tangle'...
2,Kiss of Steel London Steampunk 1,"McMaster, Bec",Honoria Todd,"{ ""gender"": 1, ""year"": 2012, ""genres"": [""paran...",['https://www.goodreads.com/book/show/13507512...
3,The Jerusalem Inception A young talmudic Schol...,"Azrieli, Avraham",Lottie,"{ ""gender"": 0, ""year"": 2011, ""genres"": [""histo...",['https://azrielibooks.com/the-jerusalem-incep...
4,The Circle Trilogy The Circle Trilogy 13,"Roberts, Nora",Glenna,"{ ""gender"": 1, ""year"": 2006, ""genres"": [""fanta...",['https://hollis.biblionix.com/catalog/titles/...
...,...,...,...,...,...
95,Covet The Clann 2,"Darnell, Melissa",Savannah Colbert,"{""gender"":1,""year"":2012,""genres"":[""young adul...",['https://www.goodreads.com/book/show/13548152...
96,Reason to Breathe Breathing 1,"Donovan, Rebecca",Emma Thomas,"{ ""gender"": 1, ""year"": 2011, ""genres"": [""young...",['https://www.goodreads.com/book/show/11561469...
97,The Magic Finger,"Dahl, Roald",NaN,"{""gender"": 0, ""year"": 1966, ""genres"": [""child...",['https://www.gradesaver.com/the-magic-finger/...
98,The Time of My Life,"Ahern, Cecelia",Lucy Silchester,"{ ""gender"": 1, ""year"": 2011, ""genres"": [""magic...",['https://www.goodreads.com/book/show/10893138...


In [40]:
df_result['answer_parsed'] = df_result['answer'].apply(json.loads)
df_answers = pd.json_normalize(df_result['answer_parsed'])


In [41]:
df_final = pd.concat([df_result[['title', 'author', 'fmc_name', 'citation']], df_answers], axis=1)
df_final

,title,author,fmc_name,citation,gender,year,genres,feature_1,feature_2,feature_3,...,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26
0,Its first loveJust like the last one,"Garg, Sachin",Kanika Merchant,['https://www.bookchor.com/book/9788192222660/...,0,0,"[romance, young adult]",0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Vipers Tangle,"Mauriac, François",Isa Fondaudège,['https://www.enotes.com/topics/vipers-tangle'...,0,1932,"[literary fiction, catholic literature, psycho...",0,0,0,...,0,0,1,0,1,0,1,0,0,0
2,Kiss of Steel London Steampunk 1,"McMaster, Bec",Honoria Todd,['https://www.goodreads.com/book/show/13507512...,1,2012,"[paranormal romance, steampunk, fantasy]",1,0,1,...,1,1,0,1,0,1,1,0,1,0
3,The Jerusalem Inception A young talmudic Schol...,"Azrieli, Avraham",Lottie,['https://azrielibooks.com/the-jerusalem-incep...,0,2011,"[historical fiction, spy thriller]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Circle Trilogy The Circle Trilogy 13,"Roberts, Nora",Glenna,['https://hollis.biblionix.com/catalog/titles/...,1,2006,"[fantasy, romance]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Covet The Clann 2,"Darnell, Melissa",Savannah Colbert,['https://www.goodreads.com/book/show/13548152...,1,2012,"[young adult, fantasy, paranormal romance]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,Reason to Breathe Breathing 1,"Donovan, Rebecca",Emma Thomas,['https://www.goodreads.com/book/show/11561469...,1,2011,"[young adult, romance, drama]",1,0,1,...,0,1,1,1,0,1,0,0,0,0
97,The Magic Finger,"Dahl, Roald",NaN,['https://www.gradesaver.com/the-magic-finger/...,0,1966,"[children's literature, fantasy]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,The Time of My Life,"Ahern, Cecelia",Lucy Silchester,['https://www.goodreads.com/book/show/10893138...,1,2011,"[magical realism, chick lit, contemporary]",0,0,0,...,1,1,1,0,1,1,1,0,1,0


In [45]:
# Check for missings in any of the rows 
df_final.isnull().sum()


title         0
author        0
fmc_name      1
citation      0
gender        0
year          0
genres        0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
feature_11    0
feature_12    0
feature_13    0
feature_14    0
feature_15    0
feature_16    0
feature_17    0
feature_18    0
feature_19    0
feature_20    0
feature_21    0
feature_22    0
feature_23    0
feature_24    0
feature_25    0
feature_26    0
dtype: int64

## Round 1
Start: $45.21
After 10: $45.18
So ... for 14.400

## Round 2
Start: $45.18
After 10: $45.12
So ... for 14.400

## Round 3
Start: $45.12
After 100: $44.74

----------------------------------------------------

In [ ]:
# Reponses column is string, convert to list
df_result["answer"] = df_result["answer"].apply(ast.literal_eval)

In [ ]:
def validate_response_format(row):
    r = row['answer']

    # Check that it's a list and has length 29
    if not isinstance(r, list) or len(r) != 29:
        return False

    try:
        # r[0] should be 0 or 1
        if r[0] not in [0, 1]:
            return False

        # r[1] should be an int
        if not isinstance(r[1], int):
            return False

        # r[2] should be a list of strings
        if not isinstance(r[2], list):
            return False
        if not all(isinstance(tag, str) for tag in r[2]):
            return False

        # r[3:] should all be 0 or 1
        if not all(isinstance(x, int) and x in [0, 1] for x in r[3:]):
            return False

    except Exception as e:
        print(f"⚠️ Issue in row: {row['title']} — Error: {e}")
        return False

    return True

df_result['is_valid'] = df_result.apply(validate_response_format, axis=1)
invalid_rows = df_result[~df_result['is_valid']]
print(f"Number of invalid rows: {len(invalid_rows)}")
print(invalid_rows[['title', 'answer']])


In [ ]:
df_result['answer'].iloc[91]

In [ ]:
df_result.iloc[91]

### Cost Overview  
**Round 1**
* started with $5.03
* ran for 77 books
* left with $4.83
* so $0.2 for 77 books
* so $0.26 for 100 books
* We'll have around 80% of the original 18027 books, so ~14.421 books
* so around $37,45 for all books 
  
**Round 2**
* started with $4.12
* ran for 77 books
* left with $3.63
* so $0.49 for 77 books 
* so around $91,82 for 14421 books

In [ ]:
# Save data frame
df_fmc.to_csv('df_fmc_features.csv', index=False)